# Вычисление параметров молекулярной механики
Дана структура этана в виде Z-matrix. Задача расчитать константы расстяжения связей, изменения валентных и торсионных углов

In [ ]:
import psi4
import numpy as np
psi4.core.set_output_file('output.dat')

In [ ]:
t= """
C
C 1  %3.5f
H 1  1.08439 2 111.200
H 1  1.08439 2 111.200 3 120
H 1  1.08439 2 111.200 3 -120
H 2  1.08439 1 111.200 3 180
H 2  1.08439 1 111.200 3 60
H 2  1.08439 1 111.200 3 -60
"""
# Мы можем поочередно изменять значения длин связей углов
bonds =  np.linspace(1,2,30)
geos = [ t % b for b in bonds ]

In [ ]:
ener =[]
for g in geos:
    m = psi4.geometry(g)
    #try:
    psi4.set_options({"maxiter": 200, "fail_on_maxiter" :  True})
    ener.append(psi4.energy('scf/cc-pvtz', molecule = m ))

Апроксимируем результаты квантово-химического расчета с помощью аналитических функций

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import optimize

x_o=bonds
y_o=ener

fitfunc = lambda p, x: p[0]*pow(p[1]-x,2) + p[2] # Укажите что за функция
errfunc = lambda p, x, y: fitfunc(p, x) - y

p0 = [1,1, -79]
p1, success = optimize.leastsq(errfunc, p0[:], args=(x_o, y_o))
print ("Optimized params:", p1)

plt.plot(x_o, y_o, "ro", x_o,fitfunc(p1,x_o),"r-",c='blue',alpha=0.5)
plt.xlim(1,2.1)
plt.show()

In [ ]:
fitfunc = lambda p, x: p[0]*1/(np.exp(p[1]*x+p[2]))+p[3]
errfunc = lambda p, x, y: fitfunc(p, x) - y

p0 = [1,1,1, -79]
p1, success = optimize.leastsq(errfunc, p0[:], args=(x_o, y_o))
print ("Optimized params:", p1)

plt.plot(x_o, y_o, "ro", x_o,fitfunc(p1,x_o),"r-",c='blue',alpha=0.5)
plt.title('New length vs Energie')
plt.xlabel('new length')
plt.ylabel('energie')
plt.show()

## Задание
* Необходимо проделать тоже самое для валентного и торионного угла
* Сравните результаты с статьей http://ambermd.org/antechamber/gaff.pdf

In [ ]:
fitfunc = lambda p, x: p[0]*(np.sin(p[1]*x))+p[2]*(np.cos(p[3]*x))+p[4] #Функция для апроксимации торсионного угла

## Расчет зарядов

In [ ]:
import psiresp

In [ ]:
ET_mol = psiresp.Molecule.from_smiles(
    ET,  optimize_geometry=True,
    conformer_generation_options=dict(n_max_conformers=5, keep_original_conformer=False),
)

In [ ]:
geometry_options = psiresp.QMGeometryOptimizationOptions(
    method="b3lyp", basis="6-31g*")
esp_options = psiresp.QMEnergyOptions(
    method="b3lyp", basis="6-31g*",
)

In [ ]:
job = psiresp.Job(molecules=[ET_mol], working_directory="ET_working", qm_optimization_options=geometry_options, qm_esp_options=esp_options, n_processes=12)
job.run()

In [ ]:
%%bash

cd ET_working/optimization && bash run_optimization.sh

In [ ]:
job = psiresp.Job(molecules=[ET_mol], working_directory="ET_working", qm_optimization_options=geometry_options, qm_esp_options=esp_options, n_processes=12)
job.run()

In [ ]:
%%bash

cd ET_working/single_point && bash run_single_point.sh

In [ ]:
job = psiresp.Job(molecules=[ET_mol], working_directory="ET_working", qm_optimization_options=geometry_options, qm_esp_options=esp_options, n_processes=12)
job.run()

* В отчете представьте молекулу с частичными зарядами